# Final Image & Caption joint training

https://gist.github.com/elkhand/412f9dc4cd1a72c4571354e81c93d695

In [79]:
# Imports for Caption model

import os, sys, io,re, string, pathlib, random
from collections import Counter, OrderedDict

import hecutils.data_utils as dt
import hecutils.scoring_utils as sc
import hecutils.plotting_utils as pt
import hecutils.caption_utils as caput
import hecutils.image_utils as imut


import tensorflow as tf
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.layers import Dense, Conv1D, MaxPooling1D, GRU, Bidirectional, LSTM
from keras.layers.core import Dropout, Flatten, Masking, ActivityRegularization
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import plot_model
from keras import regularizers
from keras.preprocessing import text
from keras.layers.embeddings import Embedding

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.utils import shuffle, class_weight
from sklearn.model_selection import StratifiedKFold, train_test_split

import numpy as np
import pandas as pd


from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

import matplotlib.pyplot as plt
import time

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline

################################################################################################################
# Imports for Image model

import os, sys, re, string, pathlib, random, io, time, glob
from collections import Counter, OrderedDict
from shutil import copyfile, rmtree

#import hecutils.resnet152 as resnet
from hecutils.resnet152 import ResNet152

import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords

import hecutils.data_utils as dt
import hecutils.scoring_utils as sc
import hecutils.plotting_utils as pt
import hecutils.image_utils as imut



import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.layers import Concatenate, MaxPooling2D, Conv2D, ZeroPadding2D, merge, Input, GRU, Bidirectional, LSTM, MaxPooling1D, Conv1D,Dense, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers.core import Dropout, Flatten, Masking, ActivityRegularization
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import text
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras import optimizers
from keras.regularizers import l2

# from fastText import load_model
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors


import numpy as np
import pandas as pd
from sklearn.utils import shuffle, class_weight
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix

from os import listdir
from os.path import isfile, join


%matplotlib inline 

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set Keras to use Tensorflow GPU in the backend

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

num_cores = 8
GPU=True
CPU = not GPU

if GPU:
    num_GPU = 1
    num_CPU = 8
if CPU:
    num_CPU = 8
    num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

## Global variables

In [3]:
# Caption model
# fix random seed for reproducibility
seed = 7

oasis_csv_path = "dataset/metadata/OASIS.csv"
captions_root = "/home/elkhand/git-repos/human-emotions-classifier/dataset/metadata"
captions_root_path = pathlib.Path(captions_root)
human_output_caption_to_label_csv_path = captions_root_path/'humanCaptionWithLabeldf.csv'
human_caption_csv_path = captions_root_path/'captions.csv'
#fasttext_embedding_path = 'embedding/wiki-news-300d-1M.vec'
fasttext_embedding_path = '/home/elkhand/datasets/glove-vectors/glove.twitter.27B.200d.txt'
model_results_root_dir = "model/"
inputDataset_csv_path = captions_root_path/"inputDataset.csv"
testDataset_csv_path = captions_root_path/"testDataset.csv"

neutralLow = 3.0 
neutralHigh = 5.0

auto_output_caption_to_label_csv_path = captions_root_path/'autoCaptionWithLabeldf.csv'
auto_caption_csv_path = captions_root_path/'auto_generated_captions.csv'


dataset_path = human_output_caption_to_label_csv_path
# dataset_path = auto_output_caption_to_label_csv_path

kfold_splits = 7 # 10 # 7 # 5 # 10 # 7 
test_size = 0.1

embedding_dimension = 200 # 300
hidden_layer_dim = 32
batch_size = 16 # 64
nb_epochs = 100
dropout = 0.3
recurrent_dropout=  0.6
patience = 10
verbose = 1

useF1Score = False # True

################################################################################################################
# Image model

oasis_csv_path = "dataset/metadata/OASIS.csv"
oasis_images_src = "dataset/images/"
input_images_src = "dataset/input-joint/"
test_images_src = "dataset/test-joint/"
model_results_root_dir = "img_model-joint/"

input_images_classified = "dataset/input-classified-joint/"
test_images_classified = "dataset/test-classified-joint/"

# ou can downlaod weights here: https://gist.github.com/flyyufelix/7e2eafb149f72f4d38dd661882c554a6
weights_path = "/home/elkhand/weights/resnet152_weights_tf.h5"

dataset_groups=["train", "val"]
classes = ["negative", "neutral", "positive"]

# fix random seed for reproducibility
seed = 7

test_size = 0.1

kfold_splits =  7 #5

neutralLow = 3.0
neutralHigh = 5.0

nb_epochs = 100
patience = 10 # ReduceLROnPlateau has 5
batch_size = 32 # 32  

FC_SIZE = 128 # 1024
LAYERS_TO_UNFREEZE = 10

img_height = 224 # 299
img_width = 224  # 299

useF1Score = False
verbose=1 



## Create <caption,label> CSV files

In [4]:
dt.create_caption_to_label(oasis_csv_path,human_caption_csv_path, human_output_caption_to_label_csv_path,neutralLow, neutralHigh)
dt.create_caption_to_label(oasis_csv_path,auto_caption_csv_path, auto_output_caption_to_label_csv_path,neutralLow, neutralHigh)

## Divide data into train/val/test datasets

Read dataframe to have:

<imageName, caption, label>

1. Read into df <imageId, label>
2. Then separate data into input and test datasets

In [35]:
dfImageIdCaptionLabel = pd.read_csv(dataset_path, header=0, sep="|")
dfImageIdCaptionLabel.columns = ["id","caption", "label"]
dfImageIdCaptionLabel["caption"] = dfImageIdCaptionLabel["caption"].apply(lambda x: " ".join(caput.get_words_withoutstopwords(x.lower().split())))
#dfImageIdCaptionLabel["label"] = dfImageIdCaptionLabel["label"].apply(lambda x: caput.change_label_str_to_int(x))


dfImageIdImageName = dt.get_image_id_to_image_title_as_df(oasis_csv_path)
dfImageIdImageName.columns = ['id', 'image_name']
dfImageIdImageName['image_name'] = dfImageIdImageName['image_name'].apply(lambda x: x + ".jpg") 
printCnt = 5
# has [id, caption, label]
df = pd.merge(dfImageIdCaptionLabel, dfImageIdImageName, on="id")
#print(df.head(printCnt))



input_x, test_x, input_y,  test_y = train_test_split(df["id"],
                                                     df["label"],
                                                     test_size=test_size,
                                                     random_state=seed,
                                                     stratify=df["label"])

inputDataset = pd.concat([input_x, input_y], axis=1)
testDataset = pd.concat([test_x, test_y], axis=1)

inputDataset = inputDataset.dropna()
testDataset = testDataset.dropna()
inputDataset = inputDataset.reset_index()
testDataset = testDataset.reset_index()

# print("inputDataset\n", inputDataset.head(10))
# print("testDataset\n", testDataset.head(10))

print("\n")
print("Label distribution in inputDataset", inputDataset.groupby('label').label.count())
print("Label distribution in testDataset", testDataset.groupby('label').label.count())


inputData = df.loc[df['id'].isin(inputDataset.id)]
testData = df.loc[df['id'].isin(testDataset.id)]

# print("inputData\n", inputData.head())
# print("testData\n", testData.head())

inputIds = set(inputData['id'].values)
testIds = set(testData['id'].values)

print("Input data size", len(inputIds))
print("Test data size", len(testIds))

for inputId in inputIds:
    if inputId in testIds:
        raise inputId + " inputId exists both in test and input dataset"
        
for testId in testIds:
    if testId in inputIds:
        raise testId + " testId exists both in test and input dataset"        

inputData.head()



Label distribution in inputDataset label
negative    147
neutral     378
positive    285
Name: label, dtype: int64
Label distribution in testDataset label
negative    16
neutral     42
positive    32
Name: label, dtype: int64
Input data size 810
Test data size 90


,id,caption,label,image_name
0,I1,two acorns lying ground next oak leaves.,neutral,Acorns 1.jpg
2,I100,ruined walls church backdrop white clouds blue...,neutral,Building 2.jpg
3,I101,man free fall attached blue bungee jumping app...,neutral,Bungee jumping 1.jpg
4,I102,falling man attached bungee jumping apparatus....,neutral,Bungee jumping 2.jpg
6,I104,man kneeling front tent two similar-looking gi...,positive,Camping 1.jpg


## Creating test and input dataset, and `positive,neutral,negative` under each dataset

In [56]:
# Delete input images dir
rmtree(input_images_src, ignore_errors=True)
os.makedirs(input_images_src)


# Delete test images dir
rmtree(test_images_src, ignore_errors=True)
os.makedirs(test_images_src)




# Copy input images into input dir, and test images into test dir
imut.copy_imgs_into(oasis_images_src, inputData['image_name'], input_images_src)
imut.copy_imgs_into(oasis_images_src, testData['image_name'], test_images_src)

# Divide input images into train and dev set, and each one into {negative, neutral, positive}
isForTest = False
X_train = inputData['image_name']
y_train = inputData['label']
dt.create_dataset("train", input_images_src, input_images_classified, X_train, y_train, isForTest)
#X_val = inputData['image_name'] # TODO COrrect
#y_val = inputData['label']
#dt.create_dataset("val", input_images_src, input_images_classified, X_val, y_val, isForTest)

## Loading Fasttext Embeddings

You can download fasttext word vectors from here:

https://fasttext.cc/docs/en/english-vectors.html

https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip


In [7]:
def load_embedding(path):
    word2vec = {}
    with io.open(path, encoding="utf8") as f:
        for line in f:
            entries = line.rstrip().split(" ")
            word, entries = entries[0], entries[1:]
            word2vec[word] = np.array(entries).astype(np.float) # Convert String type to float
    print('embedding size : %d' % len(word2vec))
    print('embedding dimension : %s' % (word2vec['apple'].shape,))
    return word2vec
    
wordToVec = {}
wordToVec = load_embedding(fasttext_embedding_path)



embedding size : 1193514
embedding dimension : (200,)


## Joint model, which will learn both from images and captions together

In [32]:
def build_model(max_seq_len, num_of_classes, config): 
    #model = Sequential()
    max_words = 5000
    text_inputs = Input(shape=(None, config['embedding_dimension']))
    masking = Masking(mask_value=0., )(text_inputs) #input_shape=(None, config['embedding_dimension'])
    lstm1 = LSTM(max_seq_len, return_sequences=True, dropout=config['dropout'], recurrent_dropout=config['recurrent_dropout'])(text_inputs)
    branch_1 = LSTM(max_seq_len, dropout=config['dropout'], recurrent_dropout=config['recurrent_dropout'])(lstm1)
    
    # Image input branch - a pre-trained Inception module followed by an added fully connected layer
    base_model = InceptionV3(weights='imagenet', include_top=False)
    # Freeze Inception's weights - we don't want to train these
    for layer in base_model.layers:
        layer.trainable = False

    # add a fully connected layer after Inception - we do want to train these
    branch_2 = base_model.output
    branch_2 = GlobalAveragePooling2D()(branch_2)
    branch_2 = Dense(1024, activation='relu')(branch_2)

    # merge the text input branch and the image input branch and add another fully connected layer
    joint = merge([branch_1, branch_2], mode='concat')
    joint = Dense(512, activation='relu')(joint)
    joint = Dropout(0.5)(joint)
    predictions = Dense(num_of_classes, activation='softmax')(joint)
    
    
    if config['useF1Score']:
        metrics = ['accuracy', sc.f1, sc.recall, sc.precision]
    else:
        metrics = ['accuracy']
        
    full_model = Model(inputs=[base_model.input, text_inputs], outputs=[predictions])

    full_model.compile(loss='categorical_crossentropy',
                   optimizer='adam', # 'rmsprop'
                   metrics=metrics)
    print(full_model.summary())
    return full_model


def get_config():
    conf = {}
    conf["kfold_splits"] = kfold_splits
    conf["batch_size"] = batch_size
    conf['embedding_dimension'] = embedding_dimension
    conf['recurrent_dropout'] = recurrent_dropout
    conf['dropout'] = dropout
    conf["nb_epochs"] = nb_epochs
    conf['useF1Score'] = useF1Score
    conf['verbose'] = verbose
    conf['useF1Score'] = useF1Score
    conf['seed'] = seed
    conf["img_height"] = img_height
    conf["img_width"] = img_width
    conf["kfold_splits"] = kfold_splits
    conf["batch_size"] = batch_size
    conf["nb_epochs"] = nb_epochs
    conf['useF1Score'] = useF1Score
    conf['verbose'] = verbose
    return conf 

#full_model = build_model(40, 3, get_config())




In [98]:
#isForTrain = True
#trainDir = input_images_classified + "/" + "train" + "/"
#train_batches = imut.get_data_generator(trainDir, get_config(), isForTrain)  

print(train_batches)
print(inputData.head())



#image_names, image_labels = dt.get_image_name_and_label(oasis_csv_path, neutralLow, neutralHigh)
#image_names = np.array(image_names)
#image_labels = np.array(image_labels)
print(image_names[:2])
print("input_images_src", input_images_src)

X_train = []
for img_name in inputData['image_name']:
    fpath = input_images_src + img_name
    img = load_img(fpath, target_size=(224,224))
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    X_train.append(x)
    
print(X_train[:2])

     id                                            caption     label  \
0    I1           two acorns lying ground next oak leaves.   neutral   
2  I100  ruined walls church backdrop white clouds blue...   neutral   
3  I101  man free fall attached blue bungee jumping app...   neutral   
4  I102  falling man attached bungee jumping apparatus....   neutral   
6  I104  man kneeling front tent two similar-looking gi...  positive   

             image_name  
0          Acorns 1.jpg  
2        Building 2.jpg  
3  Bungee jumping 1.jpg  
4  Bungee jumping 2.jpg  
6         Camping 1.jpg  
['Acorns 1.jpg' 'Alcohol 7.jpg']
input_images_src dataset/input-joint/
[array([[[176., 155., 174.],
        [166., 148., 162.],
        [163., 145., 171.],
        ...,
        [245., 244., 213.],
        [237., 248., 208.],
        [255., 247., 218.]],

       [[171., 148., 168.],
        [178., 156., 179.],
        [163., 144., 164.],
        ...,
        [250., 247., 214.],
        [251., 252., 218.],
   

In [120]:
X = inputData["caption"]
y = inputData["label"]
max_seq_len = int(inputData['caption'].map(lambda x: caput.get_non_stop_word_count(x.split())).max())


dfTrain = pd.concat([X, y], axis=1)
dfTrain.columns = ['caption', 'label']

class_to_index = {}
index_to_class = {}

print(inputData.head(15))

cnt = 0
for index, row in dfTrain.iterrows():
    print(index, row['caption'],row['label'])
    cnt +=1
    if cnt >15:
        break

print(dfTrain.head(15))        

X_train_text, y_train_index, num_of_classes, class_to_index, index_to_class = \
            caput.load_dataset_StratifiedKFold(
                            dfTrain,
                            wordToVec, 
                            max_seq_len, 
                            class_to_index, 
                            index_to_class,
                            get_config())
y_train = caput.convert_index_to_one_hot(y_train_index, num_of_classes) 
        
#y_ints = [y.argmax() for y in y_train]
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_ints),
#                                                  y_ints)
#print("class_weights", class_weights)
print("class_to_index", class_to_index)
print(X_train_text[:2])
print(type(X_train_text))

0 two acorns lying ground next oak leaves. neutral
2 ruined walls church backdrop white clouds blue sky. neutral
3 man free fall attached blue bungee jumping apparatus. river mountain him. wearing white shirt blue pants. neutral
4 falling man attached bungee jumping apparatus. wearing white shirt black sunglasses. neutral
6 man kneeling front tent two similar-looking girls sides. hands rest shoulder, one top other. positive
7 group young students classroom knees hands folded. neutral
9 man boy playing board game camping site tabletop supported logs tied together. positive
10 man wearing blue jacket headlamp lighting fire pile wood kindling front him. neutral
11 three young boys older boy blue white canoe. three young ones orange lifevests on. positive
12 two glasses champagne bread cheese plate, outdoors front body water pier. positive
15 tent glowing light inside, pitched snow field snow covered mountains background. neutral
16 assembly lit orange cream candles different sizes set tab

In [116]:
epochs = 2
print(y_train[:3])
y_train = np.array(y_train)
print(y_train[:2])
print(type(y_train))
history = full_model.fit([np.array(X_train), np.array(X_train_text)], np.array(y_train),
                                   batch_size=batch_size,
                                   epochs=100,
                                   shuffle=True,
                                   validation_split=0.2,
                                   verbose=1) 

#,  validation_split=0.2,epochs=epochs,
#  steps_per_epoch=  2000 // get_config()['batch_size'],

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [1. 0. 0.]]
<class 'numpy.ndarray'>
Train on 648 samples, validate on 162 samples
Epoch 1/100
648/648 [==============================] - 6s 9ms/step - loss: 0.6353 - acc: 0.7315 - val_loss: 11.4253 - val_acc: 0.2840
Epoch 2/100
648/648 [==============================] - 3s 4ms/step - loss: 0.5296 - acc: 0.7716 - val_loss: 11.5764 - val_acc: 0.2778
Epoch 3/100
648/648 [==============================] - 3s 4ms/step - loss: 0.5136 - acc: 0.7793 - val_loss: 11.3108 - val_acc: 0.2840
Epoch 4/100
648/648 [==============================] - 3s 5ms/step - loss: 0.4505 - acc: 0.8225 - val_loss: 11.5415 - val_acc: 0.2840
Epoch 5/100
648/648 [==============================] - 3s 4ms/step - loss: 0.4452 - acc: 0.8133 - val_loss: 11.5414 - val_acc: 0.2840
Epoch 6/100
648/648 [==============================] - 3s 5ms/step - loss: 0.3434 - acc: 0.8580 - val_loss: 11.3471 - val_acc: 0.2901
Epoch 7/100
648/648 [==============================] - 3s 4ms/st

KeyboardInterrupt: 